In [37]:
import json
from collections import Counter

In [9]:
json_dir = '../json_data/'
with open('../json_data/WebQA_test.json','r') as f:
    test_data = json.load(f)
with open('../json_data/WebQA_train_val.json','r') as f:
    train_data = json.load(f)

In [10]:
print(Counter([train_data[k]['split'] for k in train_data]))
print(len(set([train_data[k]['Guid'] for k in train_data])))
print(Counter([train_data[k]['Qcate'] for k in train_data]))

Counter({'train': 36766, 'val': 4966})
41732
Counter({'text': 20267, 'YesNo': 7320, 'Others': 5412, 'choose': 4220, 'number': 2118, 'color': 1830, 'shape': 565})


In [4]:
test_data['d5bbc4440dba11ecb1e81171463288e9']

{'Q': '"Are both the Original Playboy Mansion and Gage Park High School made of brick?"',
 'A': [''],
 'split': 'test',
 'Guid': 'd5bbc4440dba11ecb1e81171463288e9',
 'txt_Facts': [{'title': 'Gage Park High School',
   'fact': 'Gage Park High School opened in 1939. Beginning in the late–1960s, racial tensions grew between black and white students at the school.',
   'url': 'https://en.wikipedia.org/wiki/Gage_Park_High_School',
   'snippet_id': 'd5bbc4440dba11ecb1e81171463288e9_0'},
  {'title': 'Playboy Mansion',
   'fact': "Hefner established the original Playboy Mansion in 1959. It was a 70-room brick and limestone residence in Chicago's Gold Coast, which had been built in 1899.",
   'url': 'https://en.wikipedia.org/wiki/Playboy_Mansion',
   'snippet_id': 'd5bbc4440dba11ecb1e81171463288e9_1'},
  {'title': 'Chicago Lawn, Chicago',
   'fact': 'Most of the community is zoned to Gage Park High School. Some sections are zoned to Harper High School, Hubbard High School, and another high scho

### Todos
- [x] category statistics
- [x] making up subset

#### Category Statistics

In [11]:
len(train_data)

41732

In [19]:
qcate2count = dict(Counter([train_data[k]['Qcate'] for k in train_data]))
del qcate2count['text']

In [20]:
qcate2count

{'YesNo': 7320,
 'Others': 5412,
 'number': 2118,
 'color': 1830,
 'choose': 4220,
 'shape': 565}

In [21]:
counts = qcate2count.values()
ratios = [count/sum(counts) for count in counts]
qcate2ratio = dict(zip(list(qcate2count.keys()), ratios))

In [22]:

qcate2ratio

{'YesNo': 0.34102026554856746,
 'Others': 0.25213137665967855,
 'number': 0.0986722571628232,
 'color': 0.08525506638714186,
 'choose': 0.19659911483810855,
 'shape': 0.02632191940368041}

In [24]:
subsetSize = 10000
qcate2hc = {qcate:int(subsetSize*ratio) for (qcate,ratio) in qcate2ratio.items()}

In [27]:
qcate2hc = {'YesNo':2500, 'Others':2000, 'number':1500, 'color':1500, 'choose':2000, 'shape':1000}

In [56]:
subWebqa = {}
import collections
from tqdm import tqdm
qcate2items = collections.defaultdict(list)
for k in tqdm(train_data):
    if train_data[k]['Qcate'] in qcate2hc:
        qcate2items[train_data[k]['Qcate']].append((k, train_data[k]))
from random import shuffle


100%|██████████| 41732/41732 [00:00<00:00, 1078230.44it/s]


In [57]:
qcate2hc

{'YesNo': 2500,
 'Others': 2000,
 'number': 1500,
 'color': 1500,
 'choose': 2000,
 'shape': 1000}

In [58]:
import random
random.seed(11777)
for qcate in qcate2items:
    currPoolChoices = random.sample(qcate2items[qcate], min(len(qcate2items[qcate]), qcate2hc[qcate]))
    print(len(currPoolChoices))
    for k, v in currPoolChoices:
        subWebqa[k] = v

2500
2000
1500
1500
2000
565


In [59]:
len(subWebqa)

10065

In [60]:
print(Counter([subWebqa[k]['split'] for k in subWebqa]))
print(len(set([subWebqa[k]['Guid'] for k in subWebqa])))
print(Counter([subWebqa[k]['Qcate'] for k in subWebqa]))

Counter({'train': 8880, 'val': 1185})
10065
Counter({'YesNo': 2500, 'Others': 2000, 'choose': 2000, 'number': 1500, 'color': 1500, 'shape': 565})


In [49]:
len(random.choices(qcate2items['YesNo'], k = 2000))

2000

#### Make test_data subset

In [61]:
test_qcate2count = dict(Counter([test_data[k]['Qcate'] for k in test_data]))
del test_qcate2count['text']

KeyError: 'Qcate'

In [65]:
len(test_data)

7540

In [67]:
testSubWebqa = {k:v for k,v in list(random.sample(test_data.items() ,2003))}

/tmp/ipykernel_64752/3513286519.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  testSubWebqa = {k:v for k,v in list(random.sample(test_data.items() ,2003))}


In [70]:
len(testSubWebqa)

2003

In [71]:
# testSubWebqa['d5c9dd040dba11ecb1e81171463288e9']

In [72]:
# test_data['d5bbc4440dba11ecb1e81171463288e9']

In [76]:
train_subset = {k:v for k,v in subWebqa.items() if subWebqa[k]['split'] == 'train'}
valid_subset = {k:v for k,v in subWebqa.items() if subWebqa[k]['split'] == 'val'}

In [77]:
len(train_subset)

8880

In [78]:
len(valid_subset)

1185

In [79]:
len(testSubWebqa)

2003

In [80]:
print(Counter([train_subset[k]['split'] for k in train_subset]))
print(len(set([train_subset[k]['Guid'] for k in train_subset])))
print(Counter([train_subset[k]['Qcate'] for k in train_subset]))

Counter({'train': 8880})
8880
Counter({'YesNo': 2218, 'choose': 1755, 'Others': 1747, 'color': 1351, 'number': 1318, 'shape': 491})


In [81]:
print(Counter([valid_subset[k]['split'] for k in valid_subset]))
print(len(set([valid_subset[k]['Guid'] for k in valid_subset])))
print(Counter([valid_subset[k]['Qcate'] for k in valid_subset]))

Counter({'val': 1185})
1185
Counter({'YesNo': 282, 'Others': 253, 'choose': 245, 'number': 182, 'color': 149, 'shape': 74})


In [82]:
with open('../subWebqa/data/train_subWebqa.json','w')as f:
    json.dump(train_subset, f)
with open('../subWebqa/data/val_subWebqa.json','w') as f:
    json.dump(valid_subset, f)
with open('../subWebqa/data/test_subWebqa.json','w') as f:
    json.dump(testSubWebqa, f)